# Byonoy Absorbance 96 Automate

| Summary                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Photo                                            |
|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------|
| - [OEM Link](https://byonoy.com/absorbance-automate/)<br>- **Communication Protocol / Hardware**: Serial (HID)/ USB-A<br>- **Communication Level**: Firmware<br>- VID:PID 16d0:1199 <br>- Serial: "BYOMAA00058" <br>-  | ![quadrants](img/byonoy_absorbance_96_automate.png) |

---
## Setup Instructions (Physical)

The Byonoy Absorbance 96 Automate is a an absorbance plate reader consisting of...
1. a `base` containing the liqht source,
2. a `reader_cap` containing the light detectors, and
3. a `cap_adapter` representing a simple resource_holder for the `reader_cap`

It requires only one cable connections to be operational:
1. USB cable (USB-C at `base` end; USB-A at control PC end)

---
## Setup Instructions (Programmatic)

If used with a liquid handler, first setup the liquid handler:

In [1]:
from pylabrobot.liquid_handling import LiquidHandler, LiquidHandlerChatterboxBackend
from pylabrobot.resources import STARDeck

lh = LiquidHandler(deck=STARDeck(), backend=LiquidHandlerChatterboxBackend())

In [2]:
await lh.setup()

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash was assigned to the liquid handler.
Resource trash_core96 was assigned to the liquid handler.
Resource waste_block was assigned to the liquid handler.


Then generate a plate definition for the plate you want to read:

In [3]:
from pylabrobot.resources.coordinate import Coordinate
from pylabrobot.resources.cellvis.plates import CellVis_96_wellplate_350uL_Fb


plate = CellVis_96_wellplate_350uL_Fb(name='plate')
lh.deck.assign_child_resource(plate, location=Coordinate(0, 0, 0))

Resource plate was assigned to the liquid handler.


Now instantiate the Byonoy absorbance plate reader:

In [4]:
from pylabrobot.plate_reading.byonoy import (
    byonoy_absorbance_adapter,
    byonoy_absorbance96_base_and_reader
)

cap_adapter = byonoy_absorbance_adapter(name='cap_adapter')

base, reader_cap = byonoy_absorbance96_base_and_reader(name='base', assign=True)

lh.deck.assign_child_resource(cap_adapter, location=Coordinate(400, 0, 0))

Resource cap_adapter was assigned to the liquid handler.


In [5]:
await reader_cap.setup()

reader_cap.setup_finished

True

In [6]:
cap_adapter, base, reader_cap

(ResourceHolder(name='cap_adapter', location=Coordinate(400.000, 000.000, 000.000), size_x=127.76, size_y=85.59, size_z=14.07, category=resource_holder),
 ByonoyBase(name='base_base', location=None, size_x=138, size_y=95.7, size_z=27.7, category=None),
 PlateReader(name='base_reader', location=Coordinate(000.000, 000.000, 010.660), size_x=138, size_y=95.7, size_z=0, category=None))

---

## Usage / Machine Features

### Query Machine Configuration

In [7]:
await reader_cap.backend.get_available_absorbance_wavelengths()

CancelledError: 

### Measure Absorbance

In [9]:
await reader_cap.backend.read_absorbance(
    plate = CellVis_96_wellplate_350uL_Fb,
    wavelength = 600 # units: nm
)

TimeoutError: Reading luminescence data timed out after 2 minutes.

## Disconnect from Reader

In [11]:
await reader_cap.stop()